In [1]:
import pandas as pd


In [15]:
import csv

def process_headline(headline):
    processed = headline.replace('Â\xa0', ' ').replace('â\x80\x98', "'")
    if ';;;' in processed:
        processed = processed.split(';;;')[0]
    return processed

# Assuming the CSV file is named 'data.csv' and has the correct format
file_name = 'filteredDataBGlobe.tsv'

userIDsTrain = []
articleHeadlinesTrain = []
datesTrain = []

userIDsValidation = []
articleHeadlinesValidation = []
datesValidation = []

userIDsTest = []
articleHeadlinesTest = []
datesTest = []

with open(file_name, mode='r', encoding='utf-8') as file:
    size = 0
    tsv_reader = csv.reader(file, delimiter='\t')
    next(tsv_reader)  
    for row in tsv_reader:
        size += 1
        userID, headline, date = row[0], row[1], row[2]
        if headline.strip(): 
            if (date[0:4] == "2017"):
                userIDsTrain.append(userID)
                processed = process_headline(headline)
                articleHeadlinesTrain.append(processed)
                datesTrain.append(date)
            elif (date[0:4] == "2018" and int(date[5:7]) < 7 ) : 
                userIDsValidation.append(userID)
                processed = process_headline(headline)
                articleHeadlinesValidation.append(processed)
                datesValidation.append(date)
            else : 
                userIDsTest.append(userID)
                processed = process_headline(headline)
                articleHeadlinesTest.append(processed)
                datesTest.append(date)
    print (size)

71099615


In [19]:
print(len(userIDsTest))

16931101


In [ ]:
print (user_ids[1000])

In [ ]:
print(article_headlines[10])

In [ ]:
print(consumption_times[10])

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sentence_transformers import SentenceTransformer
import random

NameError: name 'userIDs' is not defined

In [45]:
import csv
from collections import Counter


def filter_data(userIDs, articleHeadlines, dates, limit):
    user_dates = {}
    for user_id, date in zip(userIDs, dates):
        if user_id not in user_dates:
            user_dates[user_id] = set()
        if date not in user_dates[user_id]:
            user_dates[user_id].add(date)

    multiple_dates_users = {user for user, dates in user_dates.items() if len(dates) > 1}

    filtered_data = []
    unique_user_ids = set()

    for user_id, headline, date in zip(userIDs, articleHeadlines, dates):
        if user_id in multiple_dates_users and user_id not in unique_user_ids and len(unique_user_ids) < limit:
            unique_user_ids.add(user_id)
        if user_id in unique_user_ids and user_id in multiple_dates_users:
            filtered_data.append((user_id, headline, date))

    return filtered_data
    

def append_to_tsv(filtered_data, filename):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:  # 'a' mode for appending
        tsv_writer = csv.writer(file, delimiter='\t')
        for user_id, headline, time in filtered_data:
            tsv_writer.writerow([user_id, headline, time])

combined_filename = 'combined_data_80k.tsv'

filtered_data_train = filter_data(userIDsTrain, articleHeadlinesTrain, datesTrain, 40000)
append_to_tsv(filtered_data_train, combined_filename)

filtered_data_validation = filter_data(userIDsValidation, articleHeadlinesValidation, datesValidation, 20000)
append_to_tsv(filtered_data_validation, combined_filename)

filtered_data_test = filter_data(userIDsTest, articleHeadlinesTest, datesTest, 20000)
append_to_tsv(filtered_data_test, combined_filename)

In [46]:
def process_data(filtered_data):
    combined_article_headlines = {}
    combined_consumption_times = {}

    # Extract unique user IDs
    multiple_occurrence_users = set(user_id for user_id, _, _ in filtered_data)

    for user_id, headline, time in filtered_data:
        if user_id not in combined_article_headlines:
            combined_article_headlines[user_id] = []
            combined_consumption_times[user_id] = []

        if time not in combined_consumption_times[user_id]:
            combined_consumption_times[user_id].append(time)
            combined_article_headlines[user_id].append([headline])
        else:
            index = combined_consumption_times[user_id].index(time)
            combined_article_headlines[user_id][index].append(headline)

    return multiple_occurrence_users, combined_article_headlines, combined_consumption_times

multiple_occurrence_users_train, combined_article_headlines_train, combined_consumption_times_train = process_data(filtered_data_train)
multiple_occurrence_users_validation, combined_article_headlines_validation, combined_consumption_times_validation = process_data(filtered_data_validation)
multiple_occurrence_users_test, combined_article_headlines_test, combined_consumption_times_test = process_data(filtered_data_test)

In [47]:
print(len(multiple_occurrence_users_test))

20000


In [120]:
import torch
from torch.nn.functional import pad

def pad_and_stack(tensor_list):
    max_rows = max(tensor.size(0) for tensor in tensor_list)
    max_cols = max(tensor.size(1) for tensor in tensor_list)

    padded_tensors = []
    for tensor in tensor_list:
        row_padding = max_rows - tensor.size(0)
        col_padding = max_cols - tensor.size(1)

        padded_tensor = pad(tensor, (0, col_padding, 0, row_padding))

        padded_tensors.append(padded_tensor)

    stacked_tensor = torch.stack(padded_tensors)

    return stacked_tensor

def pad_to_max(*tensors):
    if (tensors[0].dim() == 2) : 
        max_rows = max(tensor.size(0) for tensor in tensors)
        max_cols = max(tensor.size(1) for tensor in tensors)

        # Pad each tensor and store them in a new list
        padded_tensors = []
        for tensor in tensors:
            row_padding = max_rows - tensor.size(0)
            col_padding = max_cols - tensor.size(1)

            # Pad the tensor (pad format: [pad_left, pad_right, pad_top, pad_bottom])
            padded_tensor = F.pad(tensor, (0, col_padding, 0, row_padding))

            padded_tensors.append(padded_tensor)

        return tuple(padded_tensors)
    
    
    max_rows = max(tensor.size(0) for tensor in tensors)
    max_cols = max(tensor.size(1) for tensor in tensors)
    max_depth = max(tensor.size(2) for tensor in tensors)  # Add this line

    # Pad each tensor and store them in a new list
    padded_tensors = []
    for tensor in tensors:
        row_padding = max_rows - tensor.size(0)
        col_padding = max_cols - tensor.size(1)
        depth_padding = max_depth - tensor.size(2)  # Add this line

        padded_tensor = F.pad(tensor, (0, depth_padding, 0, col_padding, 0, row_padding))

        padded_tensors.append(padded_tensor)

    return tuple(padded_tensors)
    
from datetime import datetime, timedelta

def days_between_dates(date_str, end_date_str):
    date_format = "%Y-%m-%d"
    start_date = datetime.strptime(date_str, date_format)
    end_date = datetime.strptime(end_date_str, date_format)
    return (end_date - start_date).days

In [121]:
print(len(multiple_occurrence_users_train))

40000


In [131]:
class UserEncoder(nn.Module):
    def __init__(self, embedding_dim, lstm_hidden_dim, mlp_hidden_dim, lambda_val, alpha, beta):
        super(UserEncoder, self).__init__()
        
        self.sentence_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
        self.lstm = nn.LSTM(lstm_hidden_dim*2, lstm_hidden_dim, batch_first=True)
        
        self.mlp = nn.Sequential(
            nn.Linear(embedding_dim, mlp_hidden_dim),
            nn.ReLU(),
            nn.Linear(mlp_hidden_dim, embedding_dim)
        )
        
        self.lambda_val = lambda_val
        self.alpha = alpha
        self.beta = beta

    def forward(self, user_ids, article_headlines, consumption_times, curr_time):
                
        all_embeddings = [torch.tensor(self.sentence_model.encode(headlines)) for headlines in article_headlines]
        max_articles = max([emb.shape[0] for emb in all_embeddings])
    
        # Pad the embeddings to have the same size
        padded_embeddings = []
        for emb in all_embeddings:
            pad_size = max_articles - emb.shape[0]
            if pad_size > 0:
                pad = torch.zeros(pad_size, emb.shape[1])
                padded_emb = torch.cat([emb, pad], dim=0)
            else:
                padded_emb = emb
            padded_embeddings.append(padded_emb)

        
        all_embeddings = torch.stack(padded_embeddings)
        
        date_format = "%Y-%m-%d"
        weighted_sum = torch.zeros_like(all_embeddings[0])
        for (times, embedding) in zip(consumption_times, all_embeddings) : 
            current_date_range = days_between_dates(times, curr_time)
            weight = torch.exp(-self.lambda_val * torch.tensor(current_date_range))
            weighted_sum += (weight * embedding)
                
        output = self.mlp(weighted_sum)
        p_consistent = output[:, :output.shape[1]// 2]
        p_transients = output[:, output.shape[1]//2:]
        #lstm_out_consistent, _ = self.lstm(p_consistents)
        #lstm_out_transient, _ = self.lstm(p_transients)

        #updated_p_consistent = lstm_out_consistent[-1]
        #updated_p_transient = lstm_out_transient[-1]

        return p_consistent, p_transients
    
    def use_lstm(self, con, tran) : 
        combined = torch.cat((con, tran), dim=1).unsqueeze(0)
        lstm_output, _ = self.lstm(combined)
        lstm_output = lstm_output.squeeze(0)

        new_p_consistent = lstm_output[:, :lstm_output.shape[1]// 2]
        new_p_transient = lstm_output[:, lstm_output.shape[1]//2:]
        
        return new_p_consistent

    def generate_positive_embeddings(self, user_ids, headlines, consumption_times, curr_time):
        random_idx = random.randint(0, len(headlines) - 1)
        selected_headlines = headlines[random_idx]
        selected_times = consumption_times[random_idx]

        subsampled_headlines = random.sample(selected_headlines, len(selected_headlines) // 2)
        if len(subsampled_headlines) > 1:
            p_u_con, p_u_tra = self.forward(user_ids, [subsampled_headlines], [selected_times], curr_time)
        else:
            p_u_con, p_u_tra = self.forward(user_ids, [selected_headlines], [selected_times], curr_time)

        return torch.cat((p_u_con, p_u_tra), dim=1)

    def generate_negative_embeddings(self, target_embedding, all_embeddings):

        max_distance = -1
        negative_embedding_idx = -1

        for idx, emb in enumerate(all_embeddings):
            if torch.equal(target_embedding, emb):
                continue

            distance = torch.norm(target_embedding - emb, p=2)

            if distance > max_distance:
                max_distance = distance
                negative_embedding_idx = idx

        return all_embeddings[negative_embedding_idx] if negative_embedding_idx >= 0 else all_embeddings[0]


    def compute_loss(self, p_u, p_positive, p_negative, p_u_con, p_u_tra):
        listOfTensors = pad_to_max(p_u, p_positive, p_negative)
        p_u = listOfTensors[0]
        p_positive = listOfTensors[1]
        p_negative = listOfTensors[2] 
        contrastive_loss = torch.relu(torch.norm(p_u - p_positive, p=2)**2 - 
                              torch.norm(p_u - p_negative, p=2)**2 + 
                              self.alpha)
        
        if torch.all(contrastive_loss <= 0):
            return 0
        
        lstmVal = self.use_lstm(p_u_con, p_u_tra)
        listOfTensors2 = pad_to_max(lstmVal, p_u_con)
        
        lstmVal = listOfTensors2[0]
        p_u_con = listOfTensors2[1]
        
        consistency_loss = self.beta * torch.norm(lstmVal - p_u_con, p=2, dim=1)**2
        
        
        return torch.mean(contrastive_loss + consistency_loss)


    def training_step(self, users, combined_article_headlines, combined_consumption_times, ttv):
        if (ttv == "train") : 
            startDate = "2017-01-01"
            date = "2017-12-31"
        if (ttv == "test") :
            startDate = "2018-01-01"
            date = "2018-06-30"
        if (ttv == "validation") : 
            startDate = "2018-07-01"
            date = "2018-12-31"
            
        
        
        date = datetime.strptime(date, "%Y-%m-%d")
        currDate = startDate
        currDate = datetime.strptime(currDate, "%Y-%m-%d")
        
        
        loss = 0 
        
        while currDate < date : 
            batch_p = []
            batch_p_cons = []
            batch_p_transient = []
            batch_p_pos = []
            batch_p_negatives = []
            currDate = currDate.strftime("%Y-%m-%d")
            for user_id in users:
                
                headlines_sequence = combined_article_headlines[user_id]
                times_sequence = combined_consumption_times[user_id]                

                p_consistent, p_transient = self.forward(user_id, headlines_sequence, times_sequence, currDate)
                p_pos = self.generate_positive_embeddings(user_id, headlines_sequence, times_sequence, currDate)
                p = torch.cat((p_consistent, p_transient), dim=1)
                
                
                batch_p_cons.append(p_consistent)
                batch_p_transient.append(p_transient)
                batch_p.append(p)
                batch_p_pos.append(p_pos)

            batch_p_cons = pad_and_stack(batch_p_cons)
            batch_p_pos = pad_and_stack(batch_p_pos)
            batch_p = pad_and_stack(batch_p)
            batch_p_transient = pad_and_stack(batch_p_transient)

            for value in batch_p:
                p_negative = self.generate_negative_embeddings(value, batch_p)
                batch_p_negatives.append(p_negative)
            
            #compute_loss(self, p_u, p_positive, p_negative, p_u_con, p_u_tra):
            
            for input1, input2, input3, input4, input5 in zip (batch_p, batch_p_pos, batch_p_negatives, batch_p_cons, batch_p_transient) : 
                currLoss = self.compute_loss(input1, input2, input3, input4, input5)
                loss += currLoss
            
            currDate = datetime.strptime(currDate, "%Y-%m-%d")
            currDate = currDate + timedelta(days=1)
            
                
        return loss

In [132]:
from torch.optim import lr_scheduler

def train(model, user_ids, article_headlines, consumption_times, optimizer, num_epochs=8, batch_size=128):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
    model.train()
    
    batch_user_ids = user_ids[:256]
    
    batch_consumption_times = consumption_times
    batch_article_headlines = article_headlines
    
    scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)
    
    for epoch in range(num_epochs):
        total_loss = 0
        num_batches = 0
        
        for i in range(0, len(batch_user_ids), batch_size):
            mini_batch_user_ids = batch_user_ids[i:i+batch_size]
            mini_batch_consumption_times = batch_consumption_times
            mini_batch_article_headlines = batch_article_headlines
            
            optimizer.zero_grad()
            loss = model.training_step(mini_batch_user_ids, mini_batch_article_headlines, mini_batch_consumption_times, "train")
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            num_batches += 1
            torch.cuda.empty_cache()
        
        scheduler.step()
        # Calculate and print the average loss per batch
        avg_loss = total_loss / num_batches
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}, Learning Rate: {scheduler.get_last_lr()[0]}")

# Assuming the model and data are defined
embedding_dim = 768
lstm_hidden_dim = 384
mlp_hidden_dim = 384
lambda_val = 0.5
alpha = 0.5
beta = 0.5

model = UserEncoder(embedding_dim, lstm_hidden_dim, mlp_hidden_dim, lambda_val, alpha, beta)
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [ ]:
mu_users = list(multiple_occurrence_users_train)
train(model,mu_users , combined_article_headlines_train, combined_consumption_times_train, optimizer)

In [ ]:
torch.save(model.state_dict(), 'encoderFinal2.pth')

In [ ]:
headlines_sequence = []
times_sequence = []
user_ids_list = ['U102704']
user_ids2_list = ['U207812']
headlines_sequence_2 = [['Luxury store sets minimum spend for Santa encounters', 'University athlete meets tragic fate in practice session', 'Dairy giant Dean Foods goes bankrupt', 'Lawsuit against firearm maker gets green light from Supreme Court', 'Child succumbs to gunshot wound', 'College freshman dies after incident at fraternity house', 'Evidence of Turkish Forces targeting civilians captured by U.S. Drones', 'Seahawks triumph in nail-biting overtime against 49ers'], ['Parent brings baby to drug transaction, child gets injured', 'Prominent Solar Panel Manufacturer Shuts Down Production', 'For Sale: Deserted Missile Complex in Arizona listed at $400k', 'Firefighter loses life in line of duty in Massachusetts', 'Valiant Officer Pulls Driver From Flaming Wreckage', 'McLaren Reveals the Elva: A Windshield-less Hypercar Priced at $1.7 Million', 'Universities in Hong Kong under siege as police crackdown intensifies']]


headlines_sequence = combined_article_headlines['U102704']
print (headlines_sequence)
print (headlines_sequence_2)
times_sequence = combined_consumption_times[user_ids_list[0]]

In [ ]:
with torch.no_grad():  # Disable gradient computation during inference for efficiency
    predictions = model(user_ids_list, headlines_sequence, times_sequence)
    predictions2 = model(user_ids2_list, headlines_sequence_2, times_sequence)

In [ ]:
print(len(predictions2))

In [ ]:
#print (multiple_occurrence_users[3501])
print (predictions[0][7])


In [ ]:
difference = predictions2 - predictions

In [ ]:
difference_tuple = tuple(tensor2 - tensor1 for tensor1, tensor2 in zip(predictions2, predictions))

In [ ]:
print(difference_tuple[1][6])

In [ ]:
predict_consumption_times = [combined_consumption_times[user_id] for user_id in predict_user_ids]
predict_article_headlines = [combined_article_headlines[user_id] for user_id in predict_user_ids]

In [ ]:
print(predict_consumption_times)

In [ ]:
are_identical = all(torch.equal(tensor1, tensor2) for tensor1, tensor2 in zip(predictions, predictions2))

print("Tuples are identical:", are_identical)

In [ ]:
total_difference = sum(torch.abs(tensor2 - tensor1).sum() for tensor1, tensor2 in zip(predictions2, predictions))

In [ ]:
print(total_difference)

In [ ]:
user_ids_list = ['U102704']
user_ids2_list = ['U207812']
headlines_sequence_2 = [['Luxury store sets minimum spend for Santa encounters', 'University athlete meets tragic fate in practice session', 'Dairy giant Dean Foods goes bankrupt', 'Lawsuit against firearm maker gets green light from Supreme Court', 'Child succumbs to gunshot wound', 'College freshman dies after incident at fraternity house', 'Evidence of Turkish Forces targeting civilians captured by U.S. Drones', 'Seahawks triumph in nail-biting overtime against 49ers'], ['Parent brings baby to drug transaction, child gets injured', 'Prominent Solar Panel Manufacturer Shuts Down Production', 'For Sale: Deserted Missile Complex in Arizona listed at $400k', 'Firefighter loses life in line of duty in Massachusetts', 'Valiant Officer Pulls Driver From Flaming Wreckage', 'McLaren Reveals the Elva: A Windshield-less Hypercar Priced at $1.7 Million', 'Universities in Hong Kong under siege as police crackdown intensifies']]


headlines_sequence = combined_article_headlines['U102704']
#print (headlines_sequence)
#print (headlines_sequence_2)
times_sequence = combined_consumption_times[user_ids_list[0]]

with torch.no_grad():  
    predictions = model(user_ids_list, headlines_sequence, times_sequence)
    predictions2 = model(user_ids2_list, headlines_sequence, times_sequence)

    are_identical = all(torch.equal(tensor1, tensor2) for tensor1, tensor2 in zip(predictions, predictions2))

print("Tuples are identical:", are_identical)

In [ ]:
headlines_sequence_3 = [['Local bakery introduces new line of vegan pastries', 'Researchers discover new species of deep-sea creatures', 'City plans to open new public library next month', 'Scientists announce breakthrough in renewable energy technology', 'Young pianist wins international music competition', 'University unveils plans for environmentally-friendly campus renovations', 'Rare bird species spotted in local wildlife reserve', '5 highlights from the latest tech expo'], ['Gardener discovers ancient artifact in backyard', 'New planet discovered in our solar system', 'Local artist transforms abandoned building into public art space', 'Veterinarian volunteers to help injured wildlife in rainforest', 'Firefighter adopts dog he rescued from burning building', 'The latest electric car model breaks records for speed and efficiency', 'International Food Festival attracts visitors from around the globe']]

with torch.no_grad():  
    predictions = model(user_ids_list, headlines_sequence, times_sequence)
    predictions2 = model(user_ids2_list, headlines_sequence_3, times_sequence)

    are_identical = all(torch.equal(tensor1, tensor2) for tensor1, tensor2 in zip(predictions, predictions2))

print("Tuples are identical:", are_identical)

total_difference = sum(torch.abs(tensor2 - tensor1).sum() for tensor1, tensor2 in zip(predictions2, predictions))

print(total_difference)

In [ ]:
times_sequence_2 = [10,11]

with torch.no_grad():
    predictions = model(user_ids_list, headlines_sequence, times_sequence)
    predictions2 = model(user_ids2_list, headlines_sequence, times_sequence)

    are_identical = all(torch.equal(tensor1, tensor2) for tensor1, tensor2 in zip(predictions, predictions2))
    
total_difference = sum(torch.abs(tensor2 - tensor1).sum() for tensor1, tensor2 in zip(predictions2, predictions))

print(total_difference)